In [498]:
import numpy as np
import random

# Game Engine

In [499]:
class Game():
    def __init__(self):
        self.board = np.zeros(16).reshape(4, 4)

    def new_piece(self):
        print("NEW PIECE")
        flattened_board = self.board.reshape(16)
        empty_spots = np.where(flattened_board == 0)[0]
        print("\tEMPTY SPOTS: ", empty_spots)
        new_spot = random.randint(0, len(empty_spots) - 1)
        print("\tNEW SPOT: ", new_spot)
        flattened_board[new_spot] = 2
        self.board = flattened_board.reshape(4, 4)

    def left(self):    
        for m in range(0, 4):
            merged = False # only allow one merge per turn, as per the rules
            
            for i in range(0, 4):
                for n in range(1, 4):
                    if not self.board[m][n] == 0 and not merged and self.board[m][n - 1] == self.board[m][n]:
                        self.board[m][n - 1], self.board[m][n] = self.board[m][n - 1] * 2, 0
                        merged = True
                        
                    if self.board[m][n - 1] == 0 and self.board[m][n] != 0:
                        self.board[m][n - 1], self.board[m][n] = self.board[m][n], 0
                
    def right(self):
        for m in range(0, 4):
            merged = False # only allow one merge per turn, as per the rules

            for i in range(0, 4):
                for n in range(2, -1, -1):
                    if not self.board[m][n] == 0 and not merged and self.board[m][n] == self.board[m][n + 1]:
                        self.board[m][n], self.board[m][n + 1] = 0, self.board[m][n] * 2
                        merged = True
                    
                    if self.board[m][n] != 0 and self.board[m][n + 1] == 0:
                        self.board[m][n], self.board[m][n + 1] = 0, self.board[m][n]



# Test new piece generation and basic movement

In [500]:
game = Game()
game.board

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [501]:
game.new_piece()
game.new_piece()

NEW PIECE
	EMPTY SPOTS:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
	NEW SPOT:  4
NEW PIECE
	EMPTY SPOTS:  [ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15]
	NEW SPOT:  10


In [502]:
game.board

array([[0., 0., 0., 0.],
       [2., 0., 0., 0.],
       [0., 0., 2., 0.],
       [0., 0., 0., 0.]])

In [503]:
game.left()

In [504]:
game.board

array([[0., 0., 0., 0.],
       [2., 0., 0., 0.],
       [2., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [505]:
game.right()

In [506]:
game.board

array([[0., 0., 0., 0.],
       [0., 0., 0., 2.],
       [0., 0., 0., 2.],
       [0., 0., 0., 0.]])

# Test Merge Left

In [507]:
game = Game()
game.board[0][0], game.board[0][3] = 2, 2
game.board

array([[2., 0., 0., 2.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [508]:
game.left()
game.board

array([[4., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

# Test merge right

In [509]:
game = Game()
game.board[0][0], game.board[0][3] = 2, 2
game.board

array([[2., 0., 0., 2.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [510]:
game.right()
game.board

array([[0., 0., 0., 4.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])